# 텍스트 벡터화 (Vectorization)

텍스트 인코딩이라고도 함

대표적인 벡터화 방법

- Bag-of-Words (BoW)
  - 단순 빈도 기반, 텍스트 내 단어 등장 횟수를 벡터로 표현
- TF-IDF
  - 단어의 빈도와 역문서 빈도를 반영하여 가중치 부여
- 단어 임베딩 (Word Embeddings)
  - Word2Vec, GloVe
  - 단어의 의미적 관계를 고려한 밀집 벡터
- 문맥 기반 임베딩 (Contextual Embeddings)
  - BERT 등
  - 문맥에 따라 동적으로 변하는 단어 벡터를 생성


## Bag-of-Words (BoW)

핵심 아이디어
- 문서 내 단어의 등장 횟수를 기반으로 벡터를 생성

특징
- 단어의 순서는 고려하지 않음
- 단순하지만, 단어의 빈도 정보를 반영할 수 있음
- 희소한 (high-dimensional sparse) 벡터를 생성하는 경우가 많음

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

# CountVectorizer를 통해 BoW 벡터 생성
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print("Feature Names:", vectorizer.get_feature_names_out())
print("Bag-of-Words Matrix:\n", X.toarray())

## TF-IDF (Term Frequency-Inverse Document Frequency)

핵심 아이디어
- 단순한 빈도수 외에도 단어가 문서 내에서 얼마나 중요한지를 반영

특징
- TF (Term Frequency)
  - 특정 단어가 문서에 얼마나 자주 등장하는지
- IDF (Inverse Document Frequency)
  - 단어가 전체 문서 집합에서 얼마나 희귀한지를 계산하여 흔한 단어의 영향력을 줄임
- 결과적으로 각 단어에 가중치를 부여하여 벡터화

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
documents = [
    "자연어 처리는 재미있다",
    "자연어 처리는 어려운 만큼 재미있다",
    "나는 자연어 처리를 공부한다",
    "데이터 분석도 공부한다"
]
# TF-IDF 벡터라이저 객체 생성
vectorizer = TfidfVectorizer()

# 문서들을 벡터화
tfidf_matrix = vectorizer.fit_transform(documents)

# 단어 목록 출력
print("단어 목록:", vectorizer.get_feature_names_out())

# TF-IDF 매트릭스 출력
print("TF-IDF 매트릭스:\n", tfidf_matrix.toarray())

## 단어 임베딩 (Word Embeddings)

핵심 아이디어
- 각 단어를 고정 길이의 밀집 벡터로 변환하며, 단어 간 의미적 유사도를 반영

대표 모델
- Word2Vec, GloVe 등

특징
- 단어 사이의 의미적 관계를 수치적으로 표현
- 단어 벡터의 차원이 일반적으로 BoW나 TF-IDF보다 훨씬 낮으며, 연산 효율성이 좋음

### Word2Vec (https://arxiv.org/abs/1301.3781)

CBOW(Continuous Bag of Words)
- 주변 단어들을 이용하여 중심 단어를 예측
- 주변 단어들의 문맥 정보를 활용

예시

```
예문: "나는 __를 좋아합니다."
주변 단어: ['나는', '를', '좋아합니다']
중심 단어: '__'
```

Skip-Gram
- 중심 단어를 이용하여 주변 단어들을 예측
- 희귀 단어에 대해 더 좋은 성능을 보임

예시

```
중심 단어: '자연어'
주변 단어: ['나는', '처리를', '공부합니다']
```

In [ ]:
# Word2Vec 라이브러리 준비
from gensim.models import Word2Vec

In [ ]:
# 데이터 준비
sentences = [
    "나는 자연어 처리를 공부합니다",
    "자연어 처리는 재미있습니다",
    "딥러닝을 이용한 자연어 처리",
    "자연어 처리와 딥러닝은 인공지능의 핵심 분야입니다"
]
# Tokenization
sentences = [x.split(" ") for x in sentences]

In [ ]:
# 모델 초기화 및 학습
model = Word2Vec(sentences, vector_size=100, window=2, min_count=1, workers=4, sg=1)

In [ ]:
# 학습된 단어 벡터 확인
word_vectors = model.wv

In [ ]:
# '자연어' 단어 벡터 확인
print("'자연어' 단어 벡터:")
print(word_vectors['자연어'])

In [ ]:
# '자연어'와 유사한 단어 찾기
similar_words = word_vectors.most_similar('자연어')
print("'자연어'와 유사한 단어들:")
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

In [ ]:
# 두 단어 간 유사도 계산
similarity = word_vectors.similarity('자연어', '처리')
print(f"'자연어'와 '처리'의 유사도: {similarity}")

### GloVe (Global Vectors for Word Representation) (https://nlp.stanford.edu/pubs/glove.pdf)

전역 통계 활용
- 동시 발생 행렬
  - 전체 말뭉치에서 단어들이 서로 몇 번 같이 등장하는지를 집계하여 동시 발생 행렬을 제작
- 전역 정보 반영
  - Word2Vec과 같이 로컬 문맥(window)만을 고려하는 방법과 달리, GloVe는 전체 코퍼스의 통계 정보를 사용하여 단어 간 관계를 학습

수학적 모델
- GloVe는 단어 벡터들의 내적이 두 단어가 함께 등장할 확률의 로그와 가까워지도록 하는 회귀 문제로 접근
- 이를 위해 가중치 함수(weighting function)를 사용하여 자주 등장하는 단어 쌍과 드물게 등장하는 단어 쌍에 대해 균형 잡힌 학습을 수행

선형 관계 포착
- 학습된 단어 벡터들은 단순한 의미적 유사도뿐만 아니라, 벡터 연산(예: "king" - "man" + "woman" ≈ "queen")과 같은 선형 관계를 잘 반영하는 특징이 있음

In [ ]:
# GloVe 벡터 다운로드 (영어)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
import gensim

# GloVe 벡터를 로드하려면 gensim의 KeyedVectors로 변환 필요
glove_file = 'glove.6B.100d.txt'
tmp_file = 'word2vec-glove.6B.100d.txt'

from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)

# 모델 로드
model = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
# 'king' 단어 벡터 확인
print("'king' 단어 벡터:")
print(model['king'])

# 유사한 단어 찾기
similar_words = model.most_similar('king')
print("'king'과 유사한 단어들:")
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

### Word2Vec과 GloVe 간의 차이점

Word2Vec
- 로컬 문맥(window-based) 정보를 사용해 단어의 주변 단어들을 예측하는 방식(예: Skip-gram, CBOW)

GloVe
- 전체 말뭉치의 동시 발생 통계를 활용해 전역적 관점에서 단어 간 관계를 학습


## 문맥 기반 임베딩 (Contextual Embeddings)

핵심 아이디어
- 단어의 의미가 문맥에 따라 달라진다는 점을 반영하여, 각 단어에 대해 문맥을 고려한 동적 벡터를 생성

대표 모델
- BERT, ELMo 등

특징
- 문장 내에서 단어의 의미를 보다 정밀하게 파악할 수 있음
- 사전 학습된 모델을 활용하여 전이학습 가능

### BERT

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# 사전 학습된 BERT 모델과 토크나이저 로드 (여기서는 'bert-base-uncased' 사용)
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 평가 모드로 전환 (추론 시 dropout 등을 비활성화)
model.eval()

# 예제 문장
text = "This is an example sentence for BERT embedding."

# 텍스트를 토큰화하고 텐서로 변환
encoded_input = tokenizer(text, return_tensors="pt")

# 모델에 입력하여 출력 얻기 (그라디언트 계산이 필요 없으므로 no_grad() 사용)
with torch.no_grad():
    outputs = model(**encoded_input)

# BERT의 출력은 last_hidden_state와 pooler_output 등으로 구성됨
# last_hidden_state: [batch_size, sequence_length, hidden_size]
# [CLS] 토큰은 항상 시퀀스의 첫 번째 토큰이므로, 이를 선택하여 문장의 대표 임베딩으로 사용
cls_embedding = outputs.last_hidden_state[:, 0, :]

print("BERT [CLS] Embedding:\n", cls_embedding)